#DATASCI W261: Machine Learning at Scale 

* **Sayantan Satpati**
* **sayantan.satpati@ischool.berkeley.edu**
* **W261**
* **Week-4**
* **Assignment-4**
* **Date of Submission: 29-SEP-2015**

#  === Week 4: mrjob & k-means ===

## HW 4.0
---

***What is MrJob? How is it different to Hadoop MapReduce?***

***What are the mapper_final(), combiner_final(), reducer_final() methods? When are they called?***


## HW 4.1
---

***What is serialization in the context of MrJob or Hadoop?***

***When it used in these frameworks?***

***What is the default serialization mode for input and outputs for MrJob?***


## HW 4.2
---

Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:

https://kdd.ics.uci.edu/databases/msweb/msweb.html
http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/

This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.

 Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:

C,"10001",10001   #Visitor id 10001
V,1000,1          #Visit by Visitor 10001 to page id 1000
V,1001,1          #Visit by Visitor 10001 to page id 1001
V,1002,1          #Visit by Visitor 10001 to page id 1002
C,"10002",10002   #Visitor id 10001
V
Note: #denotes comments
to the format:

V,1000,1,C, 10001
V,1001,1,C, 10001
V,1002,1,C, 10001

Write the python code to accomplish this.

In [17]:
%%writefile preprocess_hw42.py
#!/usr/bin/env python
import sys
import os

if len(sys.argv) < 2:
    print "No input file is passed, Aborting!!!"
    sys.exit(1)

input_file = sys.argv[1]
output_file = input_file + '.pp'

try:
    os.remove(output_file)
except OSError:
    pass

last_visitor = None
with open(input_file, 'r') as f1:
    with open(output_file, 'a') as f2:
        for line in f1:
            line = line.strip()
            tokens = line.split(",")
            if len(tokens) == 3 and tokens[0] == 'C':
                last_visitor = tokens[2]

            if len(tokens) == 3 and tokens[0] == 'V':
                out_line = '{0},C,{1}\n'.format(line,last_visitor)
                f2.write(out_line)

Overwriting preprocess_hw42.py


In [18]:
!chmod a+x preprocess_hw42.py

In [19]:
!python preprocess_hw42.py anonymous-msweb.data
!head -n 10 anonymous-msweb.data.pp

V,1000,1,C,10001
V,1001,1,C,10001
V,1002,1,C,10001
V,1001,1,C,10002
V,1003,1,C,10002
V,1001,1,C,10003
V,1003,1,C,10003
V,1004,1,C,10003
V,1005,1,C,10004
V,1006,1,C,10005


##HW 4.3
---

**Find the 5 most frequently visited pages using mrjob from the output of 4.2 (i.e., transfromed log file).**

In [72]:
%%writefile mrjob_hw43.py
from mrjob.job import MRJob
from mrjob.step import MRStep


class MRVistedPagesCount(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
            MRStep(reducer=self.reducer_find_top_5)
        ]

    def mapper(self, _, line):
        tokens = line.strip().split(",")
        yield tokens[1], 1

    def combiner(self, page_visted, counts):
        yield page_visted, sum(counts)

    def reducer(self, page_visted, counts):
        yield None, (sum(counts), page_visted)

    # discard the key; it is just None
    def reducer_find_top_5(self, _, page_visted_pairs):
        # Store all the keys into memory (Assumption: Can be loaded into memory)
        pairs = []
        for p in page_visted_pairs:
            pairs.append(p)
        pairs.sort(key=lambda x: x[0], reverse=True)
            
        for p in pairs[:5]:
            yield p[1],p[0]
            

if __name__ == '__main__':
    MRVistedPagesCount.run()

Overwriting mrjob_hw43.py


In [73]:
!chmod a+x mrjob_hw43.py

In [74]:
# Running mrjob using command line
!python mrjob_hw43.py -r local anonymous-msweb.data.pp -q

"1008"	10836
"1034"	9383
"1004"	8463
"1018"	5330
"1017"	5108


In [41]:
# Running mrjob using a driver
from mrjob_hw43 import MRVistedPagesCount
mr_job = MRVistedPagesCount(args=['-r', 'local', 'anonymous-msweb.data.pp', 'q'])
with mr_job.make_runner() as runner: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

ERROR:mrjob.local:STDERR: + __mrjob_PWD=/tmp/mrjob_hw43.cloudera.20150926.200443.882049/job_local_dir/0/mapper/0
ERROR:mrjob.local:STDERR: + exec
ERROR:mrjob.local:STDERR: + /home/cloudera/anaconda/bin/python -c 'import fcntl; fcntl.flock(9, fcntl.LOCK_EX)'
ERROR:mrjob.local:STDERR: + export PYTHONPATH=/tmp/mrjob_hw43.cloudera.20150926.200443.882049/job_local_dir/0/mapper/0/mrjob.tar.gz:
ERROR:mrjob.local:STDERR: + PYTHONPATH=/tmp/mrjob_hw43.cloudera.20150926.200443.882049/job_local_dir/0/mapper/0/mrjob.tar.gz:
ERROR:mrjob.local:STDERR: + exec
ERROR:mrjob.local:STDERR: + cd /tmp/mrjob_hw43.cloudera.20150926.200443.882049/job_local_dir/0/mapper/0
ERROR:mrjob.local:STDERR: + /home/cloudera/anaconda/bin/python mrjob_hw43.py --step-num=0 --mapper /tmp/mrjob_hw43.cloudera.20150926.200443.882049/input_part-00000
ERROR:mrjob.local:STDERR: + __mrjob_PWD=/tmp/mrjob_hw43.cloudera.20150926.200443.882049/job_local_dir/0/mapper/0
ERROR:mrjob.local:STDERR: + exec
ERROR:mrjob.local:STDERR: + /home/cl

('1008', 10836)
('1034', 9383)
('1004', 8463)
('1018', 5330)
('1017', 5108)


##HW 4.4 
---

**Find the most frequent visitor of each page using mrjob and the output of 4.2  (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.**

### Approach

1. Create a separate file with URLS, i.e with records that start with A. This will be passed to the mrjob as an additional file for joining datasets
2. Following happens in the first pass of mrjob:
    * Mapper: Emits (pageId,visitorId), 1
    * Combiner: Combines the counts
    * Reducer: Combines the counts and emits records like (p1,v1) 100 | (p1, v2), 101 | (p1,v3), 202
    * Partitioner is used to that all keys with the same pageId goes to the same reducer
3. Following happens in the second pass of mrjob:
    * Mapper: Emits (pageId,visit_count), visitorId
    * Reducer_Init: Loads the url file into a dict
    * Reducer: Emits (pageId, pageURL),(vists, vistorId)
    * Partitioner is used to that all keys with the same pageId goes to the same reducer
    * Secondary sorting (Descending Order) is done on the visit_count. This ensures that pages with highest page visits come first, before other records which has a lower page visit for the same page.
   
**Final Output Format(output_hw44.txt): (pageId, pageURL),(vists, vistorId)**

In [1]:
# Create a file with only URL(s), i.e. records starting with 'A'
!rm -v url
!grep ^A anonymous-msweb.data > url

removed `url'


In [21]:
%%writefile mrjob_hw44.py
from mrjob.job import MRJob
from mrjob.step import MRStep


class MRFrequentVisitor(MRJob):
    def steps(self):
        return [
            MRStep(mapper=self.mapper,
                   combiner=self.combiner,
                   reducer=self.reducer),
             MRStep(mapper=self.mapper_frequent_visitor,
                   reducer_init=self.reducer_frequent_visitor_init,
                   reducer=self.reducer_frequent_visitor)
        ]

    def mapper(self, _, line):
        tokens = line.strip().split(",")
        key = "{0},{1}".format(tokens[1],tokens[4])
        yield key, 1

    def combiner(self, key, counts):
        yield key, sum(counts)

    def reducer(self, key, counts):
        yield key, sum(counts)
        
    # 2nd Pass
    
    def mapper_frequent_visitor(self, key, value):
        tokens = key.strip().split(",")
        modified_key = "{0},{1}".format(tokens[0],value)
        yield modified_key, tokens[1]
     
    
    def reducer_frequent_visitor_init(self):
        # Reads the 'url' file into a Dict for displaying additional information
        self.last_page = None
        self.pageDict = {}
        with open('url','r') as f:
            for line in f:
                tokens = line.strip().split(",")
                self.pageDict[tokens[1]] = tokens[4]
                
    def reducer_frequent_visitor(self, key, values):
        tokens = key.strip().split(",")
        page = tokens[0]
        visits = int(tokens[1])
        
        if self.last_page != page:
            self.last_page = page
            # values might be a list, if there is a tie for same key => (p1, 1000), [v1,v2,v3..]
            for value in values:
                k = '{0},{1}'.format(page, 
                                    self.pageDict.get(page, 'NA').replace("\"",""))
                v = '{0},{1}'.format(visits,
                                    value)
                yield k,v

if __name__ == '__main__':
    MRFrequentVisitor.run()

Overwriting mrjob_hw44.py


In [22]:
!chmod a+x mrjob_hw44.py

In [23]:
# Running mrjob using a Hadoop Runner in local cluster
from mrjob_hw44 import MRFrequentVisitor
import os

mr_job = MRFrequentVisitor(args=['-r', 'hadoop', 
                                 '--hadoop-home', '/usr/lib/hadoop-0.20-mapreduce',
                                 '--hadoop-bin', '/usr/bin/hadoop',
                                 '--file', 'url',
                                 '--jobconf', 'stream.num.map.output.key.fields=2',
                                 '--jobconf', 'map.output.key.field.separator=,',
                                 '--jobconf', 'mapred.text.key.partitioner.options=-k1,1',
                                 '--jobconf', 'mapred.text.key.comparator.options=-k1,1 -k2,2nr',
                                 '--jobconf', 'mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
                                 '--partitioner', 'org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner',
                                 'anonymous-msweb.data.pp', '-v'])


output_file = "output_hw44.txt"
try:
    os.remove(output_file)
except OSError:
    pass

with mr_job.make_runner() as runner, open(output_file, 'a') as f: 
    runner.run()
    # stream_output: get access of the output 
    for line in runner.stream_output():
        #print mr_job.parse_output_line(line)
        f.write(line)

The have been translated as follows
 mapred.text.key.partitioner.options: mapreduce.partition.keypartitioner.options
map.output.key.field.separator: mapreduce.map.output.key.field.separator
mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options
The have been translated as follows
 mapred.text.key.partitioner.options: mapreduce.partition.keypartitioner.options
map.output.key.field.separator: mapreduce.map.output.key.field.separator
mapred.output.key.comparator.class: mapreduce.job.output.key.comparator.class
mapred.text.key.comparator.options: mapreduce.partition.keycomparator.options


In [25]:
# Output (Max page visit for all pages is 1; All ties have been reported)
!wc -l output_hw44.txt 
!head -n 10 output_hw44.txt 

98654 output_hw44.txt
"1000,/regwiz"	"1,42411"
"1000,/regwiz"	"1,42381"
"1000,/regwiz"	"1,42320"
"1000,/regwiz"	"1,42291"
"1000,/regwiz"	"1,42285"
"1000,/regwiz"	"1,42260"
"1000,/regwiz"	"1,42213"
"1000,/regwiz"	"1,42198"
"1000,/regwiz"	"1,42176"
"1000,/regwiz"	"1,42160"


##HW 4.5 
---

Here you will use a different dataset consisting of word-frequency distributions 
for 1,000 Twitter users. These Twitter users use language in very different ways,
and were classified by hand according to the criteria:

0: Human, where only basic human-human communication is observed.

1: Cyborg, where language is primarily borrowed from other sources
(e.g., jobs listings, classifieds postings, advertisements, etc...).

2: Robot, where language is formulaically derived from unrelated sources
(e.g., weather/seismology, police/fire event logs, etc...).

3: Spammer, where language is replicated to high multiplicity
(e.g., celebrity obsessions, personal promotion, etc... )

Check out the preprints of our recent research,
which spawned this dataset:

http://arxiv.org/abs/1505.04342
http://arxiv.org/abs/1508.01843

The main data lie in the accompanying file:

topUsers_Apr-Jul_2014_1000-words.txt

and are of the form:

USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,...
.
.

where

USERID = unique user identifier
CODE = 0/1/2/3 class code
TOTAL = sum of the word counts

Using this data, you will implement a 1000-dimensional K-means algorithm on the users
by their 1000-dimensional word stripes/vectors using several 
centroid initializations and values of K.

Note that each "point" is a user as represented by 1000 words, and that
word-frequency distributions are generally heavy-tailed power-laws
(often called Zipf distributions), and are very rare in the larger class
of discrete, random distributions. For each user you will have to normalize
by its "TOTAL" column. Try several parameterizations and initializations:

(A) K=4 uniform random centroid-distributions over the 1000 words
(B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
(D) K=4 "trained" centroids, determined by the sums across the classes.

and iterate until a threshold (try 0.001) is reached.
After convergence, print out a summary of the classes present in each cluster.
In particular, report the composition as measured by the total
portion of each class type (0-3) contained in each cluster,
and discuss your findings and any differences in outcomes across parts A-D.

Note that you do not have to compute the aggregated distribution or the 
class-aggregated distributions, which are rows in the auxiliary file:

topUsers_Apr-Jul_2014_1000-words_summaries.txt

### Pre-Process File By Normalizing the values

In [35]:
# Pre-Process the input by normalizing the values
from sets import Set

input_file = 'topUsers_Apr-Jul_2014_1000-words.txt'
output_file = input_file + '.pp'

try:
    os.remove(output_file)
except OSError:
    pass

values = Set()

with open(input_file, 'r') as f1, open(output_file, 'w') as f2:
    for line in f1:
        count = 0
        elements = line.strip().split(",")
        total = int(elements[2])
        for i in xrange(3, len(elements)):
            value = round(int(elements[i]) * 100.0 / total, 6)
            elements[i] = str(value)
            values.add(value)
        f2.write(",".join(elements) + "\n")
        
print 'Min: {0}'.format(min(values))
print 'Max: {0}'.format(max(values))

Min: 0.0
Max: 40.990967


### Generate Initial Centroids for different use cases
* (A) K=4 uniform random centroid-distributions over the 1000 words
* (B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
* (C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution 
* (D) K=4 "trained" centroids, determined by the sums across the classes.

In [39]:
import numpy as np

K = 4
for k in xrange(K):
    uniform_random_arr = np.random.uniform(0.000001, 40.990967, 1000)
    print uniform_random_arr

[ 15.36125006   6.66210013  15.6132139   29.19274989   2.18895731
  11.45043565  23.97293389  36.6110226    0.57023838   2.26111747
  33.80715851  16.78734315  32.48914987  10.63849928   5.51761103
   4.40107387  18.75783738   5.22497742   7.33868472  39.73473954
  27.66263448  27.86015458   6.97935717   7.05128662  31.02425561
  26.14617663  23.23060953   4.15569203   4.80109688  25.87867148
  25.38569994   8.65309647  17.51470806  26.43696817  33.29522994
  21.92971649  21.50632923  31.8707209   24.03363609  39.794059
  23.55902003  39.97570244  21.63642727   3.48294092   3.05654531
   1.51223988   3.08825647  37.75712922  16.91345731  11.66816776
  34.13099648  12.75611301  22.89329485  33.31426639  35.83345861
  16.71963813  32.80088394   3.56180866   5.09397401  22.00271166
  14.76289969  38.34212476   4.20816984  37.39443452  11.0001664
  25.53153314   7.14956147  18.99413754   4.87328699  24.73837558
  25.02619834  36.81067721  37.59711345   3.73362754  30.29377224
  27.34362247

In [26]:
%%writefile Kmeans.py
from numpy import argmin, array, random
from mrjob.job import MRJob
from mrjob.step import MRJobStep
from itertools import chain

#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    centroid_points = array(centroid_points)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

#Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new,T):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    Flag = True
    for i in Diff:
        if(i>T):
            Flag = False
            break
    return Flag

class MRKmeans(MRJob):
    centroid_points=[]
    k=3    
    def steps(self):
        return [
            MRJobStep(mapper_init = self.mapper_init, mapper=self.mapper,combiner = self.combiner,reducer=self.reducer)
               ]
    #load centroids info from file
    def mapper_init(self):
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open("Centroids.txt").readlines()]
        open('Centroids.txt', 'w').close()
    #load data and output the nearest centroid index and data point 
    def mapper(self, _, line):
        D = (map(float,line.split(',')))
        yield int(MinDist(D,self.centroid_points)), (D[0],D[1],1)
    #Combine sum of data points locally
    def combiner(self, idx, inputdata):
        sumx = sumy = num = 0
        for x,y,n in inputdata:
            num = num + n
            sumx = sumx + x
            sumy = sumy + y
        yield idx,(sumx,sumy,num)
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, idx, inputdata): 
        centroids = []
        num = [0]*self.k 
        for i in range(self.k):
            centroids.append([0,0])
        for x, y, n in inputdata:
            num[idx] = num[idx] + n
            centroids[idx][0] = centroids[idx][0] + x
            centroids[idx][1] = centroids[idx][1] + y
        centroids[idx][0] = centroids[idx][0]/num[idx]
        centroids[idx][1] = centroids[idx][1]/num[idx]
        with open('Centroids.txt', 'a') as f:
            f.writelines(str(centroids[idx][0]) + ',' + str(centroids[idx][1]) + '\n')
        yield idx,(centroids[idx][0],centroids[idx][1])
      
if __name__ == '__main__':
    MRKmeans.run()

Writing Kmeans.py


In [ ]:
from numpy import random
from Kmeans import MRKmeans, stop_criterion
mr_job = MRKmeans(args=['Kmeandata.csv'])

#Geneate initial centroids
centroid_points = []
k = 3
for i in range(k):
    centroid_points.append([random.uniform(-3,3),random.uniform(-3,3)])
with open('Centroids.txt', 'w+') as f:
        f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

# Update centroids iteratively
i = 0
while(1):
    # save previous centoids to check convergency
    centroid_points_old = centroid_points[:]
    print "iteration"+str(i)+":"
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key, value
            centroid_points[key] = value
    print "\n"
    i = i + 1
    if(stop_criterion(centroid_points_old,centroid_points,0.01)):
        break
print "Centroids\n"
print centroid_points

####  Environment 

##### Hadoop

* For the purpose of this assignment, Cloudera VM (in Mac) has been used which comes pre-packaged with CDH 4.7.
* Folder structure for input and output for HW3.1 and HW3.2

```
[cloudera@localhost wk3]$ hadoop fs -ls /user/cloudera/w261/wk3/*/input
Found 1 items
-rw-r--r--   3 cloudera cloudera    3458517 2015-09-21 13:15 /user/cloudera/w261/wk3/hw31/input/ProductPurchaseData.txt
Found 1 items
-rw-r--r--   3 cloudera cloudera    3458517 2015-09-21 13:15 /user/cloudera/w261/wk3/hw32/input/ProductPurchaseData.txt
[cloudera@localhost wk3]$ hadoop fs -ls /user/cloudera/w261/wk3/*/output
Found 3 items
-rw-r--r--   3 cloudera cloudera          0 2015-09-21 13:16 /user/cloudera/w261/wk3/hw31/output/_SUCCESS
drwxr-xr-x   - cloudera cloudera          0 2015-09-21 13:15 /user/cloudera/w261/wk3/hw31/output/_logs
-rw-r--r--   3 cloudera cloudera        116 2015-09-21 13:16 /user/cloudera/w261/wk3/hw31/output/part-00000
Found 3 items
-rw-r--r--   3 cloudera cloudera          0 2015-09-21 13:18 /user/cloudera/w261/wk3/hw32/output/_SUCCESS
drwxr-xr-x   - cloudera cloudera          0 2015-09-21 13:16 /user/cloudera/w261/wk3/hw32/output/_logs
-rw-r--r--   3 cloudera cloudera        161 2015-09-21 13:18 /user/cloudera/w261/wk3/hw32/output/part-00000
```

##### Amazon EC2

* For HW3.3, Ubuntu 1404 (Micro) VM was spinned up in Amazon EC2 Cluster (More details in that section) in order to run PyFim on the Product Data.

In [1]:
%%writefile mapper_hw31.py

#!/usr/bin/env python

import sys
import re
import itertools

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        items = re.split(r'\s', line)
               
        for i in items:
            print '%s\t%d\t%d' %(i, 1, len(items))
        
        ''' Commenting out
        #Sort the list
        items.sort()
        
        for c in itertools.combinations(items, 1):
            print '%s,%s\t%d\t%d' %(c[0], '*', 1, len(items))
            
        for c in itertools.combinations(items, 2):
            print '%s,%s\t%d' %(c[0], c[1], 1)
        
        for c in itertools.combinations(items, 3):
            print '%s,%s,%s\t%d' %(c[0], c[1], c[2], 1)
        '''
    except Exception as e:
        print e

Overwriting mapper_hw31.py


In [2]:
!chmod a+x mapper_hw31.py

In [3]:
%%writefile reducer_hw31.py
#!/usr/bin/python
import sys
import re
from sets import Set

'''
a1 1 5
a1 1 5
a1 1 6
a2 1 18
'''

itemset_1_cnt = 0
item_last = None

THRESHOLD = 100

# Statistics
# Unique Items
uniq = Set()
# Max Basket Length
max_basket_len = 0
# Total Itemset Counts for Sizes: 1
total_itemset_1 = 0

for line in sys.stdin:
    # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\s', line)
    
        item = tokens[0]
        cnt = int(tokens[1])
        basket_len = int(tokens[2])
        
        if not item_last:
            item_last = item
            
        # Basket Length
        if basket_len > max_basket_len:
            max_basket_len = basket_len
        
        # Unique Items
        uniq.add(item)
        
        if item_last != item:
            # Check whether itemset 1 exceeds the support of 100
            if itemset_1_cnt >= THRESHOLD:
                total_itemset_1 += 1
    
            item_last = item
            itemset_1_cnt = cnt
        else:
            itemset_1_cnt += cnt
                    
# Last Record
if itemset_1_cnt >= THRESHOLD:
    total_itemset_1 += 1

print '=== Statistics ==='
print 'Total Unique Items: %d' %(len(uniq))
print 'Maximum Basket Length: %d' %(max_basket_len)
print 'Total # frequent itemsets of size 1: %d' %(total_itemset_1)
        
        

Overwriting reducer_hw31.py


In [4]:
!chmod a+x reducer_hw31.py

In [8]:
'''
HW3.1. Product Recommendations
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk3/hw31/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-input /user/cloudera/w261/wk3/hw31/input/ProductPurchaseData.txt \
-output /user/cloudera/w261/wk3/hw31/output \
-file ./mapper_hw31.py \
-mapper 'python mapper_hw31.py' \
-file ./reducer_hw31.py \
-reducer 'python reducer_hw31.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk3/hw31/output/part-00000

Deleted /user/cloudera/w261/wk3/hw31/output
packageJobJar: [./mapper_hw31.py, ./reducer_hw31.py, /tmp/hadoop-cloudera/hadoop-unjar1278596197593495009/] [] /tmp/streamjob4204464395455964255.jar tmpDir=null
15/09/21 13:15:49 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/21 13:15:49 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/21 13:15:50 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/21 13:15:50 INFO streaming.StreamJob: Running job: job_201509211304_0004
15/09/21 13:15:50 INFO streaming.StreamJob: To kill this job, run:
15/09/21 13:15:50 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509211304_0004
15/09/21 13:15:50 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201509211304_0004
15/09/21 13:15:51 INFO streaming.StreamJob:  map 0%  reduce 0%
15/0

### HW3.2. (Computationally prohibitive but then again Hadoop can handle this)
---

Note: for this part the writeup will require a specific rule ordering but the program need not sort the output.

List the top 5 rules with corresponding confidence scores in decreasing order of confidence score 
for frequent (100>count) itemsets of size 2. 
A rule is of the form: 

(item1) ⇒ item2.

Fix the ordering of the rule lexicographically (left to right), 
and break ties in confidence (between rules, if any exist) 
by taking the first ones in lexicographically increasing order. 
Use Hadoop MapReduce to complete this part of the assignment; 
use a single mapper and single reducer; use a combiner if you think it will help and justify. 

In [11]:
%%writefile mapper_hw32.py

#!/usr/bin/env python

import sys
import re
import itertools

for line in sys.stdin:
    try:
        # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        items = re.split(r'\s+', line)
        #Sort the list
        items.sort()
        
        l = len(items)
        
        for i in xrange(l):
            print '%s,*\t%d' %(items[i], 1)
            for j in xrange(i+1, l):
               print '%s,%s\t%d' %(items[i], items[j], 1)
               print '%s,%s\t%d' %(items[j], items[i], 1)
    except Exception as e:
        print e

Overwriting mapper_hw32.py


In [12]:
!chmod a+x mapper_hw32.py

In [13]:
%%writefile reducer_hw32.py
#!/usr/bin/python
import sys
import re
import heapq

itemset_1_cnt = 0
itemset_2_cnt = 0

itemset_1_last = None
itemset_2_last = None

'''
a1,* 1
a1,* 1
a1,b1 1
a1,b1 1
a1,b2 1
a1,b2 1
a2,* 1
'''

THRESHOLD = 100
# Store Itemsets 2
dict = {}

for line in sys.stdin:
    # Remove leading & trailing chars
        line = line.strip()
        # Split the line by <TAB> delimeter
        tokens = re.split(r'\s+', line)
    
        # Split the key by <COMMA> delimeter
        items = tokens[0].split(",")
        i1 = items[0]
        i2 = items[1]
        
        if not itemset_1_last:
            itemset_1_last = i1
        
        if itemset_1_last != i1:
            '''
            if itemset_1_cnt >= THRESHOLD:
                confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
                print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, tokens[0], confidence)
                dict[tokens[0]] = confidence
            '''
                        
            # Reset
            itemset_1_last = i1
            itemset_1_cnt = int(tokens[1])
            itemset_2_last = None
            itemset_2_cnt = 0
        else:
            if i2 == '*':
                itemset_1_cnt += int(tokens[1])
            else:
                if itemset_2_last != tokens[0]:
                    if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
                        confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
                        #print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, itemset_2_last, confidence)
                        dict[itemset_2_last] = confidence
                    itemset_2_last = tokens[0]
                    itemset_2_cnt = int(tokens[1]) 
                else:
                    itemset_2_cnt += int(tokens[1])                    

# Last Set of Counts
if itemset_1_cnt >= THRESHOLD and itemset_2_cnt >= THRESHOLD:
    confidence = (itemset_2_cnt * 1.0) / itemset_1_cnt
    #print '[%d,%d]%s\t%f' %(itemset_1_cnt, itemset_2_cnt, itemset_2_last, confidence)
    dict[itemset_2_last] = confidence

print '=== Top 5 Confidence ==='
sorted_dict = sorted(dict.items(), key=lambda x:(-x[1], x[0]))
for j,k in sorted_dict[:5]:
    print '%s\t%f' %(j,k)
        
        
        

Overwriting reducer_hw32.py


In [14]:
!chmod a+x reducer_hw32.py

In [15]:
'''
HW3.2. Confidence Caculations
List the top 5 rules with corresponding confidence scores in decreasing order of confidence score 
for frequent (100>count) itemsets of size 2
'''

# Delete existing Output Dirs if available
!hadoop fs -rm -r -skipTrash /user/cloudera/w261/wk3/hw32/output

# Run the Hadoop Streaming Command
!hadoop jar /usr/lib/hadoop-0.20-mapreduce/contrib/streaming/hadoop-streaming-2.0.0-mr1-cdh4.7.0.jar \
-D map.output.key.field.separator=, \
-D mapred.text.key.partitioner.options=-k1,1 \
-input /user/cloudera/w261/wk3/hw32/input/ProductPurchaseData.txt \
-output /user/cloudera/w261/wk3/hw32/output \
-file ./mapper_hw32.py \
-mapper 'python mapper_hw32.py' \
-file ./reducer_hw32.py \
-reducer 'python reducer_hw32.py'

# Show Output
!hadoop fs -cat /user/cloudera/w261/wk3/hw32/output/part-00000

Deleted /user/cloudera/w261/wk3/hw32/output
packageJobJar: [./mapper_hw32.py, ./reducer_hw32.py, /tmp/hadoop-cloudera/hadoop-unjar1867291528774777589/] [] /tmp/streamjob8875451858342305964.jar tmpDir=null
15/09/21 13:16:53 WARN mapred.JobClient: Use GenericOptionsParser for parsing the arguments. Applications should implement Tool for the same.
15/09/21 13:16:53 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/21 13:16:54 INFO streaming.StreamJob: getLocalDirs(): [/tmp/hadoop-cloudera/mapred/local]
15/09/21 13:16:54 INFO streaming.StreamJob: Running job: job_201509211304_0005
15/09/21 13:16:54 INFO streaming.StreamJob: To kill this job, run:
15/09/21 13:16:54 INFO streaming.StreamJob: UNDEF/bin/hadoop job  -Dmapred.job.tracker=localhost.localdomain:8021 -kill job_201509211304_0005
15/09/21 13:16:54 INFO streaming.StreamJob: Tracking URL: http://0.0.0.0:50030/jobdetails.jsp?jobid=job_201509211304_0005
15/09/21 13:16:55 INFO streaming.StreamJob:  map 0%  reduce 0%
15/0

### HW3.3
---

Benchmark your results using the pyFIM implementation of the Apriori algorithm
(Apriori - Association Rule Induction / Frequent Item Set Mining implemented by Christian Borgelt). 
You can download pyFIM from here:***

http://www.borgelt.net/pyfim.html

Comment on the results from both implementations (your Hadoop MapReduce of apriori versus pyFIM) 
in terms of results and execution times.

#### For this part, the following steps were performed:

1. Since I was using a Mac, I spinned up a Ubuntu 1404 (Micro) VM in Amazon EC2 Cluster
2. Installed all required libraries: pip, git, ipython
3. Downloaded the fim.so file from the link & set PYTHONPATH & LD_LIBRARY_PATH
4. Ran the following command in the VM

In [16]:
!python top5pyfim.py | sort -n -r -k 3

```
FRO40251	('DAI93865',)	1.0
FRO40251	('GRO85051',)	0.999176276771
FRO40251	('GRO38636',)	0.990654205607
FRO40251	('ELE12951',)	0.990566037736
FRO40251	('DAI88079',)	0.986725663717
FRO40251	('FRO92469',)	0.983510011779
SNA82528	('DAI43868',)	0.972972972973
DAI62779	('DAI23334',)	0.954545454545
```

#### Top 5 Comparison from HW3.2 (Map/Reduce) and HW3.3 (PyFim Apriori)

##### Map/Reduce
```
DAI93865,FRO40251	1.000000
GRO85051,FRO40251	0.999176
GRO38636,FRO40251	0.990654
ELE12951,FRO40251	0.990566
DAI88079,FRO40251	0.986726
```

##### HW3.3 (PyFim Apriori)
```
FRO40251    ('DAI93865',)    1.0
FRO40251    ('GRO85051',)    0.999176276771
FRO40251    ('GRO38636',)    0.990654205607
FRO40251    ('ELE12951',)    0.990566037736
FRO40251    ('DAI88079',)    0.986725663717
```
**As it can be seen from above, both the results are identical with each other**

* The Map/Reduce approach is much more scalable, even though it took longer to execute since there is a start up cost for Hadoop Map Reduce Jobs. As the file size gets bigger, the Map Reduce would perform much better than using PyFim.

### HW3.4 (Conceptual Exercise)

Suppose that you wished to perform the Apriori algorithm once again,
though this time now with the goal of listing the top 5 rules with corresponding confidence scores 
in decreasing order of confidence score for itemsets of size 3 using Hadoop MapReduce.
A rule is now of the form: 

(item1, item2) ⇒ item3 

Recall that the Apriori algorithm is iterative for increasing itemset size,
working off of the frequent itemsets of the previous size to explore 
ONLY the NECESSARY subset of a large combinatorial space. 
Describe how you might design a framework to perform this exercise.

In particular, focus on the following:
  — map-reduce steps required
  - enumeration of item sets and filtering for frequent candidates

* In order to solve this problem a chain of Hadoop Map Reduce Jobs need to be used. The design would be something as follows:

**Input Document: D**
```
item1 item2 item3 item4 item5
item1 item2 item4 item5
item3 item4 item5
.
.
.
```

#### 1st Pass:
  * **Mapper**: Read Document line by line and emit items as (item, 1)
  * **Reducer**: Sum the items and emit only those that are > support count (Ex: s=100). At the end of the first pass the reducer file (L1) would look like the following. Since the output from Reducer is not supposed to be a big file, the number of reducer would be passed as 1 to the Hadoop Job, so that only one output file is generated from this pass.
  ```
    item1, 102
    item3, 105
  ```
  
#### 2nd Pass:
  * **Mapper**: The output file from 1st pass would be passed to the Mapper in the form of a Distributed Cache. Assuming this file is small (Typical Super Store doesn't have more than 10,000 or so frequently sold items) it is loaded into the memory of the m/c(s) runnning mappers. The mapper would load L1 into a Dictionary (Ex: key=item1, value=102 etc). It would also create a candidate generation C2 by taking all combination pairs from L1 and create another Dictionary where keys are candidate pairs;values can be anyting like '-'. C2 would look like the following:
  ```
  (item1, item2) -
  (item1, item3) -
  ```
  The mapper would then read portion of the Document D passed to it, and emit only those pairs which are present in C2 (Candidates). Output of mapper would have key like "item1,item3" and value as "1".
  * **Reducer**: Hadoop framework would make sure Reducer receives the values for the same keys. The job of the reducer would be to sum the values for same keys, and emit only those that are > support count (Ex: s=100), just like in the 1st pass. Since the output from Reducer is not supposed to be a very big file (Though it will be much bigger than the file from 1st pass), the number of reducer would be passed as 1 to the Hadoop Job, so that only one output file is generated from this pass. The output of the reducer (L2) would look like the following. 
  ```
  item1,item2 150
  item1,item3 240
  ```
  
#### 3rd Pass:
  * **Mapper**: The output file from 2nd pass would be passed to the Mapper in the form of a Distributed Cache. Assuming this file is small it is loaded into the memory, as a Dictionary, of the m/c(s) runnning mappers. The mapper would read portion of the Document D passed to it, read line by line, and emit the following as long as the line has the candidate pairs. The mapper would emit something like the following if a candidate pair is present in the Document D:
  ```
  item1,item2,* 150
  item1,item2,item4 1
  ```
  * **Reducer**: Paritioning has to be done based on the leftmost item pair (item1,item2), which would ensure all records with the same item pair would be passed to the same reducer. And secondary sort would ensure that the reducer would get to read the total (item1,item2,* 150) before it starts calculating the confidence scores. Each reducer would need to store the confidence scores in memory to be able to spit out the top 5 once the processing is done. If the number of records exceeds the memory of the reducer, then top 5 has to be recomputed every time a confidence score is calculated by the reducer.